In [ ]:
!pip install openai
!pip install langchain
!pip install tiktoken
!pip install faiss-cpu

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your key"

#1.Prompt Template

In [ ]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(model_name="text-davinci-003")

#下面的{Coding}作为替换符
#加入了Instruction，让LLM知道提问的背景。让它知道调用旅游方面的能力。
template = """
你作为工作5-10年的程序员回答如下问题。一句话概括。
我想学习{Coding}编程，需要哪几个步骤?
"""

prompt = PromptTemplate(
    #接受用户输入
    input_variables=["Coding"],
    #定义Prompt tempalte
    template=template,
)
#这里是真正的用户输入 "Python"
final_prompt = prompt.format(Coding='Python')

print (f"组合后的用户请求: {final_prompt}")

print (f"大语言模型的回应: {llm(final_prompt)}")

组合后的用户请求: 
你作为工作5-10年的程序员回答如下问题。一句话概括。
我想学习Python编程，需要哪几个步骤?


大语言模型的回应: 需要先学习Python语法和基础知识，然后熟悉各种Python库和框架，最后完成实践项目，以加深对Python编程的理解。


#2.ChatMessagePromptTemplate 通过Role实现精准回应
LangChain提供了不同类型的MessagePromptTemplate。

HumanMessagePromptTemplate：人类消息

SystemMessagePromptTemplate：系统消息（Instruction，Conexti）

AIMessagePromptTemplate：AI消息

可以使用ChatMessagePromptTemplate，它允许用户指定角色名称。

ChatMessagePromptTemplate 通过role + 模版 的方式生成prompt

In [ ]:
from langchain.prompts import ChatMessagePromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")
prompt = "我想知道如何描写{subject}"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="小学生", template=prompt)
formatted_prompt =chat_message_prompt.format(subject="人物")
prompts = [str(formatted_prompt)]
# 将格式化的提示传递给LLM模型并获取回应
response = llm.generate(prompts)

# 打印LLM模型的回应
print(response)

generations=[[Generation(text='\n\n回答：\n\n如果你想描写一个小学生，可以先着重描述他们的外表，比如他们的脸型、发型、肤色、服装等。你也可以描述他们的动作、语言、情绪和态度，以及他们的语调和表情。此外，你也可以描述小学生的性格和他们的内心世界，比如他们有什么想法和梦想。', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'prompt_tokens': 38, 'completion_tokens': 243, 'total_tokens': 281}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('d0a98e52-0c75-4a99-8fa4-349a0841a43f'))]


#3.通过MessagesPlaceholder 实现多类型Message合作
LangChain中的"MessagesPlaceholder"功能，的应用场景：

例如，一个聊天系统，用户和AI进行了多轮的对话，对很多输入的内容进行了分析。

在这种情况下，需要对上面的对话进行总结。就需要使用"MessagesPlaceholder"讲上面人类输入和AI相应的信息作为模版放入到其中，并且限制总结的字数。

In [ ]:
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.llms import OpenAI
from langchain.schema.messages import HumanMessage,AIMessage

#创建语言模型
llm = OpenAI(model_name="text-davinci-003")
#人类消息模版
human_prompt = "讲我们上面的对话通过{word_count}个字总结一下."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

#创建聊天模版
#定义变量conversation，这个是用来存放多个消息模版的
chat_prompt = ChatPromptTemplate.from_messages([MessagesPlaceholder(variable_name="conversation"), human_message_template])

#人类输入的信息
human_message = HumanMessage(content="如何写好小学作文?")

#AI消息回应
#使用硬编码，在真实的场景中可以是语言模型返回的结果，或者使保存前几次模型返回的信息。
ai_message = AIMessage(content="""\
1. 阅读广泛：阅读是提升写作技巧的关键。阅读各种文学作品、新闻报道和其他优秀的写作作品，可以扩展你的词汇量和写作风格，并为你提供灵感和观点.
2. 练习写作：写作就像是一项技能，需要不断的练习。坚持写作，并接受反馈和指导，以不断改进你的表达和结构。多样化你的写作练习，包括叙事、说明和议论等不同类型的作文.
3. 重视编辑和校对：写作并不仅仅是把思绪表达出来，而是经过多次修改和润色才能达到最佳效果。花时间编辑和校对你的作文，确保语法准确、句子流畅，同时注意段落结构和逻辑连贯\
""")

#调用MessagesPlaceholder 分别插入human_message（人类的输入）, ai_message（模型的历史回应），最后得到总结
formatted_prompt = chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count="10").to_messages()

prompts = [str(formatted_prompt)]
# 将格式化的提示传递给LLM模型并获取回应
response = llm.generate(prompts)
print(response)


generations=[[Generation(text='\n\n总结：阅读、练习、编辑校对.', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 609, 'total_tokens': 641}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('ca52c063-7de2-44bd-ad8b-6ced86bacda9'))]


#4 Partial prompt templates 模版的部分加载

LangChain支持两种方式进行部分填充：

字符串值进行部分格式化。
返回字符串值的函数进行部分格式化。


日期的函数就是函数部分格式化

##4.1 字符串方式

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# 初始化一个OpenAI模型
llm = OpenAI(model_name="text-davinci-003")

# 创建一个PromptTemplate，它需要两个输入变量："{姓名}"和"{动作}"
prompt = PromptTemplate(template="{姓名}正在{动作}", input_variables=["姓名", "动作"])

# 使用partial方法，预先填充"姓名"变量为"小明"
partial_prompt = prompt.partial(姓名="小明")

# 使用format方法，填充剩余的"动作"变量为"打篮球"，并将结果转化为字符串
formatted_prompt = str(partial_prompt.format(动作="打篮球"))

# 将格式化的提示传递给LLM模型并获取回应
response = llm.generate([formatted_prompt])

# 打印LLM模型的回应
print(response)


generations=[[Generation(text='\n\n小明一边跳起来一边把球往篮筐里扔，小心翼翼地调整身体的姿势，把球尽可能地投进篮筐。他又站起来，移动到篮下，用双手把球接住，又用身体从不同的角度把球投进篮筐，运用起跳和抛投的技巧，努力把球投进篮筐。', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 233, 'prompt_tokens': 15, 'total_tokens': 248}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('299de547-f23d-4849-bf49-63789b797fe0'))]


##4.2 函数方式

In [ ]:
from datetime import datetime
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# 初始化一个OpenAI模型
llm = OpenAI(model_name="text-davinci-003")

# 定义一个函数，返回当前的日期和时间
def _get_datetime():
    now = datetime.now()
    return now.strftime("%Y年%m月%d日, %H点%M分%S秒")

# 创建一个PromptTemplate，它需要两个输入变量："{adjective}"和"{date}"
prompt = PromptTemplate(
    template="给我讲一个关于{date}的{adjective}笑话",
    input_variables=["adjective", "date"]
)

# 使用partial方法，预先填充"date"变量为当前的日期和时间
partial_prompt = prompt.partial(date=_get_datetime)

# 使用format方法，填充剩余的"adjective"变量为"有趣"，并将结果转化为字符串
formatted_prompt = str(partial_prompt.format(adjective="有趣"))

# 将格式化的提示传递给LLM模型并获取回应
response = llm.generate([formatted_prompt])

# 打印LLM模型的回应
print(response)


generations=[[Generation(text='\n\n2023年07月17日, 07点30分32秒，一个科学家在做一次实验，他让时间停止，结果实验失败了，他发现自己只能再次停留在07点30分32秒，科学家大声喊：“我又回到这里了！”', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 171, 'prompt_tokens': 48, 'total_tokens': 219}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('8c4d08c5-8572-4ee9-933c-3db02b5b7f6e'))]


#5.组合Prompt template

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI

# 创建LLM实例
llm = OpenAI(model_name="text-davinci-003")

# 1 定义最终的提示模板
full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

# 定义"introduction"部分的提示模板
introduction_template = """你正在模仿{person}。"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# 定义"example"部分的提示模板
example_template = """这是一个互动的例子:

问: {example_q}
答: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

# 定义"start"部分的提示模板
start_template = """现在，开始真正的互动！

问: {input}
答:"""
start_prompt = PromptTemplate.from_template(start_template)

# 将所有部分的提示模板组合到一起
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

# 格式化管道提示并生成LLM的回应
formatted_prompt = pipeline_prompt.format(
    person="小马哥",
    example_q="你最喜欢的车是什么？",
    example_a="特斯拉",
    input="你最喜欢的社交媒体网站是什么？"
)
response = llm.generate([str(formatted_prompt)])

# 打印LLM模型的回应
print(response)


generations=[[Generation(text=' 我最喜欢的社交媒体网站是推特。', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 148, 'total_tokens': 182}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('887e466a-755c-4b80-8938-485088d6c81e'))]


#6.Prompt template 序列化

支持JSON和YAML两种格式。

支持在一个文件中指定所有内容，或者将不同的组件（模板、示例等）存储在不同的文件中并进行引用。

提供了一个加载提示的单一入口点

In [ ]:
from langchain.prompts import load_prompt
from langchain.llms import OpenAI


# 加载提示模板
prompt = load_prompt('few_shot_prompt_examples_in.json')

# 使用LLM生成回应
llm = OpenAI(model_name="text-davinci-003")
response = llm.generate([str(prompt.format(adjective="有趣"))])

# 打印LLM模型的回应
print(response)


generations=[[Generation(text=' 无聊', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 84, 'total_tokens': 90}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('90ab7118-ff83-4629-9903-3e2553b6ed92'))]


#7.Example Selectors


##7.1Select by length（根据Prompt 控制示例长度）


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.llms import OpenAI

# 使用LLM生成回应
llm = OpenAI(model_name="text-davinci-003")

# 这些是一些假设任务的例子，创建反义词。
examples = [
    {"input": "开心", "output": "难过"},
    {"input": "高", "output": "矮"},
    {"input": "精力充沛", "output": "无精打采"},
    {"input": "阳光明媚", "output": "阴沉"},
    {"input": "风大", "output": "风平浪静"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="输入: {input}\n输出: {output}",
)

example_selector = LengthBasedExampleSelector(
    # 这些是它可以选择的例子。
    examples=examples,
    # 这是用来格式化例子的PromptTemplate。
    example_prompt=example_prompt,
    # 这是格式化的例子应该有的最大长度。
    # 长度是通过下面的get_text_length函数来测量的。
    max_length=30,

)

dynamic_prompt = FewShotPromptTemplate(
    # 我们提供一个ExampleSelector，而不是例子。
    #限定了长度的示例选择器
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入的反义词",
    suffix="输入: {adjective}\n输出:",
    input_variables=["adjective"],
)

# 一个有小输入的例子，所以它选择了所有的例子。
print(dynamic_prompt.format(adjective="热情"))

# 你可以使用你的语言模型来生成输出
print("测试例子")
output = llm.generate([dynamic_prompt.format(adjective="热情")])
print(output)


给出每个输入的反义词

输入: 开心
输出: 难过

输入: 高
输出: 矮

输入: 精力充沛
输出: 无精打采

输入: 阳光明媚
输出: 阴沉

输入: 风大
输出: 风平浪静

输入: 热情
输出:
测试例子
generations=[[Generation(text=' 冷漠', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 185, 'completion_tokens': 6, 'prompt_tokens': 179}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('47f3b47b-d3ff-45a8-8259-97d8b6a30a66'))]


##7.2 Select by similarity（选择与Prompt 类似的示例）

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI

# 创建一个LLM模型实例
llm = OpenAI(model_name="text-davinci-003")

# 创建一个用于格式化示例的PromptTemplate实例
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="示例输入: {input}\n示例输出: {output}",
)

# 创建一个示例列表
examples = [
    {"input": "老师", "output": "教室"},
    {"input": "医生", "output": "医院"},
    {"input": "司机", "output": "汽车"},
    {"input": "树", "output": "土地"},
    {"input": "鸟", "output": "鸟巢"},
]

# 创建一个SemanticSimilarityExampleSelector实例，用于选择与输入语义相似的示例
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,  # 可供选择的示例列表
    OpenAIEmbeddings(),  # 用于生成嵌入向量的嵌入类
    FAISS,  # 向量数据库用来存放示例，为比较做准备
    k=2  # 要选择的示例数量，返回相似的示例数字为2
)

# 创建一个FewShotPromptTemplate实例
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 用于选择示例的对象
    example_prompt=example_prompt,  # 用于格式化示例的PromptTemplate实例
    prefix="给出一个事物通常所在的位置",  # 提示的前缀 instruction
    suffix="输入: {noun}\n输出:",  # 提示的后缀
    input_variables=["noun"],  # 提示将接收的输入变量
)

# 选择一个名词
my_noun = "花朵"

# 格式化提示并打印
print(similar_prompt.format(noun=my_noun))

# 使用LLM模型生成回应
response = llm.generate([str(similar_prompt.format(noun=my_noun))])
print(response)


给出一个事物通常所在的位置

示例输入: 鸟
示例输出: 鸟巢

示例输入: 树
示例输出: 土地

输入: 花朵
输出:
generations=[[Generation(text=' 花园', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 119, 'completion_tokens': 4, 'prompt_tokens': 115}, 'model_name': 'text-davinci-003'} run=[RunInfo(run_id=UUID('cf9fb91a-1f51-4df1-a36a-13ecd45d29f3'))]


#8.LLMS
LLMs是LangChain的核心组件，LangChain并不提供自己的LLMs，而是提供了一个与多种LLMs交互的标准接口。

In [ ]:
from langchain.llms import OpenAI

# 创建一个LLM模型实例
llm = OpenAI(model_name="text-davinci-003")

# 使用__call__方法生成文本
joke = llm("给我讲个笑话")
print(joke)

# 使用generate方法生成文本
#请求数组中有两个prompt ，后面通过* 5 告诉 LLM，我要对LLM发起10次请求，因此会得到10个响应。
#由于这里是例子，实际想表达的是LLM可以接受批量的prompt 请求
#批量发起prompt 请求
responses = llm.generate(["给我讲个笑话", "给我写首诗"]*5)

# 请求的次数
print(len(responses.generations))

# 输出第一个生成的文本
print(responses.generations[0])

# 输出最后一个生成的文本
print(responses.generations[1])




两个熊在森林里玩耍，一只熊说：“我累了，我要休息一会儿。”另一只熊问：“那你怎么休息？”第一只熊说：“我要睡觉！”另一只熊说：“你怎么睡觉？你没有床！”第一只熊说：“没关系，我可以睡在树上！”
10
[Generation(text='\n\n一个人去买帽子，卖家问：“要几只？”顾客答：“不要，我是来看样的。”', generation_info={'finish_reason': 'stop', 'logprobs': None})]
[Generation(text='\n\n若梦里不见你\n不曾知你离去\n心痛如刀割\n无奈泪滂沱\n\n情意犹未抹\n多少年未曾言\n心思不曾离\n等待着你归来', generation_info={'finish_reason': 'stop', 'logprobs': None})]


#9.Caching


##9.1 内存缓存

In [ ]:
from langchain.cache import InMemoryCache
import langchain
from langchain.llms import OpenAI
import time


# 设置内存缓存
langchain.llm_cache = InMemoryCache()

# 初始化一个大型语言模型
llm = OpenAI()

# 第一次预测，结果未被缓存，所以需要较长时间

start_time = time.time()  # 获取当前时间
result1 = llm.predict("给我说个笑话")
print(result1)
end_time = time.time()  # 获取当前时间
elapsed_time = end_time - start_time  # 计算经过的时间
print(f"预测花费了 {elapsed_time} seconds.")


# 第二次预测相同的输入，由于结果已经被缓存，所以预测速度会更快

start_time = time.time()  # 获取当前时间
result2 = llm.predict("给我说个笑话")
print(result2)
end_time = time.time()  # 获取当前时间
elapsed_time = end_time - start_time  # 计算经过的时间
print(f"预测花费了 {elapsed_time} seconds.")





Q: What did the fish say when it hit the wall?
A: Dam!
预测花费了 0.7900259494781494 seconds.


Q: What did the fish say when it hit the wall?
A: Dam!
预测花费了 0.0003459453582763672 seconds.


##9.2数据库缓存

In [ ]:
from langchain.cache import SQLiteCache
import time
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

start_time = time.time()  # 获取当前时间
# 第一次，它还不在缓存中，所以应该需要更长的时间
result1= llm.predict("给我讲个笑话")
print(result1)
end_time = time.time()  # 获取当前时间
elapsed_time = end_time - start_time  # 计算经过的时间
print(f"预测花费了 {elapsed_time} seconds.")

start_time = time.time()  # 获取当前时间
# 第二次，它在缓存中，时间会短一些
result2= llm.predict("给我讲个笑话")
print(result2)
end_time = time.time()  # 获取当前时间
elapsed_time = end_time - start_time  # 计算经过的时间
print(f"预测花费了 {elapsed_time} seconds.")




一位老太太去买鱼，店主问她：“你要买活鱼还是死鱼？”
老太太说：“还活着的！”店主说：“那可不行，你得先把它们杀死！”老太太恼怒地说：“这么说，你是在告诉我，我老了！”
预测花费了 12.39306902885437 seconds.


一位老太太去买鱼，店主问她：“你要买活鱼还是死鱼？”
老太太说：“还活着的！”店主说：“那可不行，你得先把它们杀死！”老太太恼怒地说：“这么说，你是在告诉我，我老了！”
预测花费了 0.0021202564239501953 seconds.


#10.FakeListLLM

In [ ]:
# 导入所需的库和模块
from langchain.llms.fake import FakeListLLM
from langchain.agents import load_tools, initialize_agent, AgentType

# 加载名为"python_repl"的工具
tools = load_tools(["python_repl"])

# 定义一个模拟的LLM响应列表
responses = ["Action: Python REPL\nAction Input: print(2 + 2)", "Final Answer: 4"]

# 创建一个FakeListLLM实例，它会按照预定义的响应列表来响应提示
llm = FakeListLLM(responses=responses)

# 使用这些工具和LLM初始化一个代理
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# 运行代理以回答问题"2 + 2是多少?"
agent.run("whats 2 + 2")




> Entering new AgentExecutor chain...
Action: Python REPL
Action Input: print(2 + 2)
Observation: Python REPL is not a valid tool, try another one.
Thought:Final Answer: 4

> Finished chain.


'4'

#11.异步调用


In [ ]:
import time
import asyncio
from langchain.llms import OpenAI



# 定义一个串行生成的函数
def generate_serially():
    llm = OpenAI(temperature=0.9)
    for _ in range(5):
        resp = llm.generate(["我是串行函数的信息"])
        print(resp.generations[0][0].text)

# 定义一个异步生成的函数
async def async_generate(llm):
    resp = await llm.agenerate(["我是异步函数的信息"])
    print(resp.generations[0][0].text)

# 定义一个并发生成的函数
async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    tasks = [async_generate(llm) for _ in range(5)]
    await asyncio.gather(*tasks)

# 计算并发执行的时间
s = time.perf_counter()
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"并发执行在 {elapsed:0.2f} 秒内完成." + "\033[0m")

# 计算串行执行的时间
s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"串行执行在 {elapsed:0.2f} 秒内完成." + "\033[0m")


#12.流式响应

In [ ]:
# 导入所需的库
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# 创建一个使用流式响应和回调处理器的 OpenAI LLM 实例
llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)

# 使用 LLM 生成一首关于气泡水的歌曲
resp = llm("给我唱首歌。")




好的，我给你唱一首《Let It Go》：

The snow glows white on the mountain tonight
Not a footprint to be seen
A kingdom of isolation, and it looks like I'm the queen
The wind is howling like this swirling storm inside
Couldn't keep it in, Heaven knows I tried

Don't let them in, don't let them see
Be the good girl you always have to be
Conceal, don't feel, don't let them know
Well, now they know

Let it go, let it go
Can't hold it back anymore
Let it go, let it go
Turn away and slam the door
I don't care what they're going to say
Let the storm rage on
The cold never bothered me anyway

It's funny how some distance makes everything seem small
And the fears that once controlled me can't get to me at all
It's time to see what I can do
To test the limits and break through
No right, no wrong, no rules for me, I'm free

Let it go, let it go
I

#13.Output parser

"setup"和"punchline"是为了解析一个笑话的结构而设置的。"setup"是笑话的开头部分，通常是一个问题，而"punchline"是笑话的结尾部分，通常是对问题的回答或者是笑话的高潮部分。

如果你想解析其他类型的文本，你可以定义其他的字段。例如，如果你想解析一个新闻报道，你可能会定义"headline"（标题）、"author"（作者）、"date"（日期）和"content"（内容）等字段。这完全取决于你想要解析的文本的结构和你的需求。

In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

model = OpenAI(model_name='text-davinci-003', temperature=0.0)

# 定义所需的数据结构。
class Joke(BaseModel):
    #定义字段
    #这里假设笑话是一问一答的方式进行的。
    setup: str = Field(description="提出一个问题")
    punchline: str = Field(description="回答，产生一个笑话")


# 设置解析器并将指令注入到提示模板中。
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    #template 中定义了两个变量：
    #一个是控制输出的format_instructions：定义两个字段和一个验证方法
    #一个是控制输入的query
    template="回应用户的查询。\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 构建一个查询，旨在提示语言模型填充数据结构。
joke_query = "说一个笑话"
input = prompt.format_prompt(query=joke_query)
output = model(input.to_string())
print(output)






{"setup": "Why did the chicken cross the road?", "punchline": "To get to the other side!"}


#14.Parser List

In [ ]:
# 导入需要的库
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# 创建一个逗号分隔的列表输出解析器
output_parser = CommaSeparatedListOutputParser()

# 获取格式化指示
format_instructions = output_parser.get_format_instructions()

# 创建一个提示模板
prompt = PromptTemplate(
    template="列出五种{subject}。\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

# 创建一个OpenAI模型实例
model = OpenAI(temperature=0)

# 使用提示模板和主题输入变量来格式化输入
_input = prompt.format(subject="冰淇淋口味")

# 使用模型实例来生成输出
output = model(_input)

# 使用输出解析器来解析输出
output_parser.parse(output)


['草莓', '抹茶', '巧克力', '香草', '杏仁']

#15.Date time parser 日期解析

In [ ]:
# 导入需要的库
from langchain.prompts import PromptTemplate
from langchain.output_parsers import DatetimeOutputParser
from langchain.chains import LLMChain
from langchain.llms import OpenAI

# 创建一个日期时间输出解析器
output_parser = DatetimeOutputParser()

# 定义一个字符串模板
template = """回答用户的问题:

{question}

{format_instructions}"""

# 创建一个提示模板
prompt = PromptTemplate.from_template(
    template,
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# 创建一个LLMChain实例
chain = LLMChain(prompt=prompt, llm=OpenAI())

# 使用LLMChain实例来运行一个问题
output = chain.run("2020 年的圣诞节是什么时候？")

# 使用输出解析器来解析输出
output_parser.parse(output)


datetime.datetime(2020, 12, 25, 0, 0)

#16.Auto-fixing parser （自动修复解析器）
`OutputFixingParser`是一个特殊的输出解析器，它的工作方式是：如果原始的解析器（在这里是`parser`）无法解析输入的字符串（在这里是`misformatted`），那么`OutputFixingParser`会使用一个语言模型（在这里是`ChatOpenAI`）来尝试“修复”输入的字符串，使其能够被原始的解析器解析。

具体来说，`OutputFixingParser`会将无法解析的字符串和解析器期望的格式指示一起作为提示传递给语言模型，然后语言模型会生成一个新的字符串，这个新的字符串应该能够被原始的解析器解析。这就是为什么`new_parser.parse(misformatted)`能够成功解析字符串的原因。

需要注意的是，这种方法并不总是能够成功。语言模型可能无法生成一个能够被原始解析器解析的字符串，或者生成的字符串可能并不符合我们的期望。因此，使用`OutputFixingParser`时需要谨慎，并且可能需要对其输出进行额外的检查或处理。

In [ ]:
# 导入需要的库
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# 定义一个名为"Actor"的数据结构，包含"name"（名字）和"film_names"（电影名字列表）两个字段
class Actor(BaseModel):
    name: str = Field(description="演员的名字")
    film_names: List[str] = Field(description="他们主演的电影的名字列表")

# 创建一个PydanticOutputParser实例，用于解析"Actor"数据结构
parser = PydanticOutputParser(pydantic_object=Actor)

# 定义一个错误格式化的字符串
misformatted = "{'name': '孙悟空', 'film_names': ['大闹天宫']}"
#misformatted = '{"name": "孙悟空", "film_names": ["大闹天宫"]}'


# 使用解析器来解析这个错误格式化的字符串
parser.parse(misformatted)


OutputParserException: ignored

In [ ]:
from langchain.output_parsers import OutputFixingParser

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())
new_parser.parse(misformatted)

Actor(name='孙悟空', film_names=['大闹天宫'])